# ML Conventional Methods for Bioactivity Prediction

## BACE - Regression | Random Splitting | MACCS Features

##### Import libraries

- Helper function: load, split dataset, generate fingerprint

- Load model from scikit-learn, torch

- Load hyperopt module for hyperparameter tuning

In [9]:
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
import numpy as np
import pandas as pd
from helper.load_dataset import load_bace_regression
from helper.preprocess import split_train_valid_test
from helper.features_ml import smi_maccs
from helper.cal_metrics import regression_metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from hyperopt import hp, tpe, fmin, Trials, space_eval
from hyperopt.pyll import scope
from tabulate import tabulate

In [10]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

### Training pipeline

##### Random Splitting - BACE Regression Task

- Load, split dataset

- Generate fingerprint, defined target

In [11]:
# Load dataset
bace_reg = load_bace_regression()

# Split dataset
train, valid, test = split_train_valid_test(bace_reg)
merge = pd.concat((train, valid))

# Generate fingerprint
train_smis = train['SMILES']
valid_smis = valid['SMILES']
test_smis = test['SMILES']
merge_smis = merge['SMILES']
X_train = [smi_maccs(smi) for smi in train_smis]
X_valid = [smi_maccs(smi) for smi in valid_smis]
X_test = [smi_maccs(smi) for smi in test_smis]
X_merge = [smi_maccs(smi) for smi in merge_smis]


# Target defined
y_train = train['pIC50']
y_valid = valid['pIC50']
y_test = test['pIC50']
y_merge = merge['pIC50']

##### Hyperparameter tuning and model training

- Pipeline: Hyperparameter tuning using valid set -> Train best params on train + valid -> Test on test set

- Models to try:

    - Support Vector Machine
    - Random Forest
    - XGBoost
    - Deep Neural Network

#### SVM

In [12]:
# Hyperparameters tuning with Hyperopt
trials = Trials()

svm_search_space = {
    'C': hp.loguniform('C', np.log(0.1), np.log(10)),
    'epsilon': hp.uniform('epsilon', 0.01, 0.2),
    'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly'])
}

def svm_objective(params):
    model = SVR(
        C=params['C'],
        epsilon=params['epsilon'], 
        kernel=params['kernel']
    )
    model.fit(X_train, y_train)
    y_valid_hat = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_valid_hat)
    return mse

best_svm_params = fmin(
    fn=svm_objective,
    space=svm_search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

best_svm_params = space_eval(svm_search_space, best_svm_params)

model = SVR(**best_svm_params)
model.fit(X_merge, y_merge)
y_train_pred = model.predict(X_merge)
y_test_pred = model.predict(X_test)

# Calculate metrics

train_metrics = regression_metrics(y_merge, y_train_pred)
test_metrics = regression_metrics(y_test, y_test_pred)

# Print

result_header = ['Metrics', 'Train', 'Test']
result_body = [
    ["RMSE", f'{train_metrics['rmse']:.4f}', f'{test_metrics['rmse']:.4f}'],
    ["R2", f'{train_metrics['r2']:.4f}', f'{test_metrics['r2']:.4f}'],
    ["MAPE", f'{train_metrics['mape']:.4f}', f'{test_metrics['mape']:.4f}'],
    ["Pearson R", f'{train_metrics['pearsonr']:.4f}', f'{test_metrics['pearsonr']:.4f}'],
    ["Spearman R", f'{train_metrics['spearmanr']:.4f}', f'{test_metrics['spearmanr']:.4f}'],
]

print('\nSVM Results:\n')
print(f'Best params: {best_svm_params}\n')
print(tabulate(result_body, headers=result_header, tablefmt='grid'))

with open('results/bace_reg_random_maccs_svm.txt', 'w') as file:
    file.write('BACE regression | Random Splitting | MACCS Features | SVM Model\n\n')
    file.write('Results:\n\n')
    file.write(f'Best params: {best_svm_params}\n\n')
    file.write(tabulate(result_body, headers=result_header, tablefmt='grid'))

100%|██████████| 100/100 [00:17<00:00,  5.69trial/s, best loss: 0.42468461671656077]

SVM Results:

Best params: {'C': 6.21050912441982, 'epsilon': 0.15806654752758129, 'kernel': 'rbf'}

+------------+---------+--------+
| Metrics    |   Train |   Test |
+============+=========+========+
| RMSE       |  0.4767 | 0.723  |
+------------+---------+--------+
| R2         |  0.8736 | 0.7132 |
+------------+---------+--------+
| MAPE       |  0.0535 | 0.0885 |
+------------+---------+--------+
| Pearson R  |  0.9359 | 0.8481 |
+------------+---------+--------+
| Spearman R |  0.9283 | 0.7814 |
+------------+---------+--------+


#### Random Forest

In [13]:
# Hyperparameters tuning with Hyperopt
n_feats = len(X_train[0])
trials = Trials()

rf_search_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 5, 100, 5)),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 20, 1)),
    'max_features': scope.int(hp.quniform('max_features', 5, int(n_feats/2), 5)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 2, 20, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 20, 1))
}

def rf_objective(params):
    model = RandomForestRegressor(
        n_estimators=params['n_estimators'], 
        max_depth=params['max_depth'], 
        max_features=params['max_features'], 
        min_samples_leaf=params['min_samples_leaf'], 
        min_samples_split=params['min_samples_split'],
        random_state=SEED,
        n_jobs=1
    )
    model.fit(X_train, y_train)
    y_valid_hat = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_valid_hat)
    return mse

best_rf_params = fmin(
    fn=rf_objective,
    space=rf_search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

best_rf_params = space_eval(rf_search_space, best_rf_params)

model = RandomForestRegressor(**best_rf_params, random_state=SEED)
model.fit(X_merge, y_merge)
y_train_pred = model.predict(X_merge)
y_test_pred = model.predict(X_test)

# Calculate metrics

train_metrics = regression_metrics(y_merge, y_train_pred)
test_metrics = regression_metrics(y_test, y_test_pred)

# Print

result_header = ['Metrics', 'Train', 'Test']
result_body = [
    ["RMSE", f'{train_metrics['rmse']:.4f}', f'{test_metrics['rmse']:.4f}'],
    ["R2", f'{train_metrics['r2']:.4f}', f'{test_metrics['r2']:.4f}'],
    ["MAPE", f'{train_metrics['mape']:.4f}', f'{test_metrics['mape']:.4f}'],
    ["Pearson R", f'{train_metrics['pearsonr']:.4f}', f'{test_metrics['pearsonr']:.4f}'],
    ["Spearman R", f'{train_metrics['spearmanr']:.4f}', f'{test_metrics['spearmanr']:.4f}'],
]

print('\nRF Results:\n')
print(f'Best params: {best_rf_params}\n')
print(tabulate(result_body, headers=result_header, tablefmt='grid'))

with open('results/bace_reg_random_maccs_rf.txt', 'w') as file:
    file.write('BACE regression | Random Splitting | MACCS Features | RF Model\n\n')
    file.write('Results:\n\n')
    file.write(f'Best params: {best_rf_params}\n\n')
    file.write(tabulate(result_body, headers=result_header, tablefmt='grid'))

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 100/100 [00:18<00:00,  5.51trial/s, best loss: 0.4627825274314585]

RF Results:

Best params: {'max_depth': 13, 'max_features': 40, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 70}

+------------+---------+--------+
| Metrics    |   Train |   Test |
+============+=========+========+
| RMSE       |  0.5955 | 0.7161 |
+------------+---------+--------+
| R2         |  0.8028 | 0.7187 |
+------------+---------+--------+
| MAPE       |  0.0761 | 0.0935 |
+------------+---------+--------+
| Pearson R  |  0.9062 | 0.8541 |
+------------+---------+--------+
| Spearman R |  0.897  | 0.7943 |
+------------+---------+--------+


#### XGBoost

In [14]:
# Hyperparameters tuning with Hyperopt
trials = Trials()

xgb_search_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 5, 300, 5)),
    'max_depth': scope.int(hp.quniform('max_depth', 1, 10, 1)),
    "min_child_weight": scope.int(hp.quniform("min_child_weight", 1, 10, 1)),
    "subsample": hp.uniform("subsample", 0.4, 1.0),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.4, 1.0),
    "reg_lambda": hp.loguniform("reg_lambda", np.log(0.00001), np.log(100)),
    "reg_alpha": hp.loguniform("reg_alpha", np.log(0.001), np.log(1000)),
    "learning_rate": hp.loguniform("learning_rate", np.log(0.0001), np.log(1.)),
    "booster": hp.choice('booster', ['gbtree', 'gblinear']),
    'tree_method': 'hist',
    'gamma': hp.uniform('gamma', 0., 5.)
}

def xgb_objective(params):
    model = XGBRegressor(
        n_estimators=params['n_estimators'], 
        max_depth=params['max_depth'], 
        min_child_weight=params['min_child_weight'], 
        subsample=params['subsample'], 
        colsample_bytree=params['colsample_bytree'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        learning_rate=params['learning_rate'],
        booster=params['booster'],
        tree_method=params['tree_method'],
        gamma=params['gamma'],
        random_state=SEED,
        verbosity=0
    )
    model.fit(X_train, y_train)
    y_valid_hat = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_valid_hat)
    return mse

best_xgb_params = fmin(
    fn=xgb_objective,
    space=xgb_search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

best_xgb_params = space_eval(xgb_search_space, best_xgb_params)

model = XGBRegressor(**best_xgb_params, random_state=SEED)
model.fit(X_merge, y_merge)
y_train_pred = model.predict(X_merge)
y_test_pred = model.predict(X_test)

# Calculate metrics

train_metrics = regression_metrics(y_merge, y_train_pred)
test_metrics = regression_metrics(y_test, y_test_pred)

# Print

result_header = ['Metrics', 'Train', 'Test']
result_body = [
    ["RMSE", f'{train_metrics['rmse']:.4f}', f'{test_metrics['rmse']:.4f}'],
    ["R2", f'{train_metrics['r2']:.4f}', f'{test_metrics['r2']:.4f}'],
    ["MAPE", f'{train_metrics['mape']:.4f}', f'{test_metrics['mape']:.4f}'],
    ["Pearson R", f'{train_metrics['pearsonr']:.4f}', f'{test_metrics['pearsonr']:.4f}'],
    ["Spearman R", f'{train_metrics['spearmanr']:.4f}', f'{test_metrics['spearmanr']:.4f}'],
]


print('\nXGBoost Results:\n')
print(f'Best params: {best_xgb_params}\n')
print(tabulate(result_body, headers=result_header, tablefmt='grid'))

with open('results/bace_reg_random_maccs_xgb.txt', 'w') as file:
    file.write('BACE regression | Random Splitting | MACCS Features | XGBoost Model\n\n')
    file.write('Results:\n\n')
    file.write(f'Best params: {best_xgb_params}\n\n')
    file.write(tabulate(result_body, headers=result_header, tablefmt='grid'))

100%|██████████| 100/100 [00:12<00:00,  8.27trial/s, best loss: 0.4518874673603972]

XGBoost Results:

Best params: {'booster': 'gbtree', 'colsample_bytree': 0.8221040218730729, 'gamma': 0.33595718480975206, 'learning_rate': 0.04076871755490012, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 295, 'reg_alpha': 0.27444380231278226, 'reg_lambda': 6.4650709807590285, 'subsample': 0.4908057658184942, 'tree_method': 'hist'}

+------------+---------+--------+
| Metrics    |   Train |   Test |
+============+=========+========+
| RMSE       |  0.4555 | 0.6888 |
+------------+---------+--------+
| R2         |  0.8846 | 0.7397 |
+------------+---------+--------+
| MAPE       |  0.0567 | 0.0882 |
+------------+---------+--------+
| Pearson R  |  0.9429 | 0.8617 |
+------------+---------+--------+
| Spearman R |  0.9341 | 0.7992 |
+------------+---------+--------+


#### ANN

In [15]:
trials = Trials()

class FCNClassifier(nn.Module):
    def __init__(
            self,
            input_dim,
            hidden_dim_1=128,
            hidden_dim_2=128,
            hidden_dim_3=128,
            hidden_dim_4=128,
            dropout_rate=0.2,
            activation='relu',
            n_layers=4
    ):
        super(FCNClassifier, self).__init__()
        
        self.n_layers = n_layers
        self.dropout_rate = dropout_rate
        self.activation = activation

        self.fc1 = nn.Linear(input_dim, hidden_dim_1, bias=True)
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2, bias=True)
        self.fc3 = nn.Linear(hidden_dim_2, hidden_dim_3, bias=True)
        self.fc4 = nn.Linear(hidden_dim_3, hidden_dim_4, bias=True)
        
        output_dims = [hidden_dim_1, hidden_dim_2, hidden_dim_3, hidden_dim_4]
        last_hidden_dim = output_dims[n_layers-1]
        self.out = nn.Linear(last_hidden_dim, 1)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):

        x = self.fc1(x)                     # Layer 1
        x = self._activation(x)
        x = self.dropout(x)
    
        if self.n_layers >= 2:              # Layer 2 (if applicable)
            x = self.fc2(x)
            x = self._activation(x)
            x = self.dropout(x)
        
        if self.n_layers >= 3:              # Layer 3 (if applicable)
            x = self.fc3(x)
            x = self._activation(x)
            x = self.dropout(x)

        if self.n_layers >= 4:              # Layer 4 (if applicable)
            x = self.fc4(x)
            x = self._activation(x)
            x = self.dropout(x)

        x = self.out(x)

        return x
    
    def _activation(self, x):
        if self.activation == 'relu':
            return F.relu(x)
        elif self.activation == 'gelu':
            return F.gelu(x)
        elif self.activation == 'elu':
            return F.elu(x)
        elif self.activation == 'selu':
            return F.selu(x)
        else:
            return F.relu(x)
    
def train_ann(
        model,
        X_train,
        y_train,
        X_valid,
        y_valid,
        learning_rate = 0.001,
        batch_size=128,
        epochs=100,
        patience=10,
        device='cpu'
):
    model = model.to(device)

    train_dataset = TensorDataset(
        torch.FloatTensor(X_train),
        torch.FloatTensor(y_train).unsqueeze(1)
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    early_stopping = True if X_valid is not None and y_valid is not None else False
    best_mse = float('inf')
    no_improvement_count = 0
    best_state = None
    best_num_epochs = 0

    for epoch in range(epochs):
        # Train
        model.train()
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        # Valid
        if early_stopping:
            model.eval()
            with torch.no_grad():
                X_valid_tensor = torch.FloatTensor(X_valid).to(device)
                y_valid_pred = model(X_valid_tensor).cpu().numpy().flatten()
                mse = mean_squared_error(y_valid, y_valid_pred)

            if mse < best_mse:
                best_mse = mse
                no_improvement_count = 0
                best_state = model.state_dict().copy()
                best_num_epochs += 1
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    break
    
    if early_stopping and best_state is not None:
        model.load_state_dict(best_state)
        return model, best_num_epochs
        
    return model, epochs

def predict_test(model, X, device='cpu'):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        predictions = model(X_tensor).cpu().numpy().flatten()

    return predictions

def ann_objective(
        params,
        X_train,
        y_train,
        X_valid,
        y_valid,
        input_dim,
        device='cpu'
):
    model = FCNClassifier(
        input_dim=input_dim,
        hidden_dim_1=params['hidden_dim_1'],
        hidden_dim_2=params['hidden_dim_2'],
        hidden_dim_3=params['hidden_dim_3'],
        hidden_dim_4=params['hidden_dim_4'],
        dropout_rate=params['dropout_rate'],
        activation=params['activation'],
        n_layers=params['n_layers']
    )

    model, best_num_epoch = train_ann(
        model,
        X_train,
        y_train,
        X_valid,
        y_valid,
        learning_rate=params['learning_rate'],
        epochs=params['epochs'],
        patience=params['patience'],
        device=device,
        batch_size=params['batch_size']
    )

    model.eval()
    with torch.no_grad():
        X_valid_tensor = torch.FloatTensor(X_valid).to(device)
        y_valid_pred = model(X_valid_tensor).cpu().numpy().flatten()
        mse = mean_squared_error(y_valid, y_valid_pred)

    return {
        'loss': mse,
        'status': 'ok',
        'best_num_epoch': best_num_epoch
    }

ann_search_space = {
    'n_layers': scope.int(hp.quniform('n_layers', 1, 4, 1)),
    'hidden_dim_1': scope.int(hp.quniform('hidden_dim_1', 32, 192, 32)),
    'hidden_dim_2': scope.int(hp.quniform('hidden_dim_2', 32, 192, 32)),
    'hidden_dim_3': scope.int(hp.quniform('hidden_dim_3', 32, 192, 32)),
    'hidden_dim_4': scope.int(hp.quniform('hidden_dim_4', 32, 192, 32)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'batch_size': scope.int(hp.quniform('batch_size', 32, 128, 16)),
    'epochs': 100,
    'patience': 10,
    'activation': hp.choice('activation', ['relu', 'selu', 'elu', 'gelu'])
}

input_dim = len(X_train[0])

def objective_fn(params):
    return ann_objective(
        params=params,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        input_dim=input_dim
    )

best_ann_params = fmin(
    fn=objective_fn,
    space=ann_search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

best_ann_params = space_eval(ann_search_space, best_ann_params)

model = FCNClassifier(
    input_dim=input_dim,
    hidden_dim_1=best_ann_params['hidden_dim_1'],
    hidden_dim_2=best_ann_params['hidden_dim_2'],
    hidden_dim_3=best_ann_params['hidden_dim_3'],
    hidden_dim_4=best_ann_params['hidden_dim_4'],
    dropout_rate=best_ann_params['dropout_rate'],
    activation=best_ann_params['activation'],
    n_layers=best_ann_params['n_layers']
)

best_trial = trials.best_trial
best_num_epochs = best_trial['result']['best_num_epoch']

model, _ = train_ann(
    model,
    X_merge,
    y_merge,
    X_valid=None,
    y_valid=None,
    learning_rate=best_ann_params['learning_rate'],
    batch_size=best_ann_params['batch_size'],
    epochs=best_num_epochs,
)

y_train_pred = predict_test(model, X_merge)
y_test_pred = predict_test(model, X_test)

train_metrics = regression_metrics(y_merge, y_train_pred)
test_metrics = regression_metrics(y_test, y_test_pred)

# Print

result_header = ['Metrics', 'Train', 'Test']
result_body = [
    ["RMSE", f'{train_metrics['rmse']:.4f}', f'{test_metrics['rmse']:.4f}'],
    ["R2", f'{train_metrics['r2']:.4f}', f'{test_metrics['r2']:.4f}'],
    ["MAPE", f'{train_metrics['mape']:.4f}', f'{test_metrics['mape']:.4f}'],
    ["Pearson R", f'{train_metrics['pearsonr']:.4f}', f'{test_metrics['pearsonr']:.4f}'],
    ["Spearman R", f'{train_metrics['spearmanr']:.4f}', f'{test_metrics['spearmanr']:.4f}'],
]

print('\nANN Results:\n')
print(f'Best params: {best_ann_params}\n')
print(tabulate(result_body, headers=result_header, tablefmt='grid'))

with open('results/bace_reg_random_maccs_ann.txt', 'w') as file:
    file.write('BACE regression | Random Splitting | MACCS Features | ANN Model\n\n')
    file.write('Results:\n\n')
    file.write(f'Best params: {best_ann_params}\n\n')
    file.write(tabulate(result_body, headers=result_header, tablefmt='grid'))

100%|██████████| 100/100 [04:05<00:00,  2.46s/trial, best loss: 0.4752807006964817]

ANN Results:

Best params: {'activation': 'gelu', 'batch_size': 80, 'dropout_rate': 0.4117861200792948, 'epochs': 100, 'hidden_dim_1': 192, 'hidden_dim_2': 192, 'hidden_dim_3': 160, 'hidden_dim_4': 96, 'learning_rate': 0.0038658482242215878, 'n_layers': 3, 'patience': 10}

+------------+---------+--------+
| Metrics    |   Train |   Test |
+============+=========+========+
| RMSE       |  0.8485 | 0.8907 |
+------------+---------+--------+
| R2         |  0.5996 | 0.5648 |
+------------+---------+--------+
| MAPE       |  0.1043 | 0.1098 |
+------------+---------+--------+
| Pearson R  |  0.8234 | 0.807  |
+------------+---------+--------+
| Spearman R |  0.7943 | 0.7499 |
+------------+---------+--------+
